In [ ]:
pip install folium

In [ ]:

pip install haversine


In [ ]:
pip install haversine

In [ ]:
import xml.etree.ElementTree as ET
import pprint
import random as rd
import folium as flMap
from haversine import haversine
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:


anonimity_threshold = 3


class UserObject:
    def __init__(self,id,lat_lon_obj,wsscr=None):
        self.id = id
        self.lat_lon_obj = lat_lon_obj
        self.wsscr=wsscr

class LatLon:
    def __init__(self,lat,lon):
        self.lat = float(lat)
        self.lon = float(lon)


class WSSCR:
    cloaked_search_space_radius= 1  # kilo meters
    def __init__(self,WSSCR_list,WSSCR_id):
        self.id = WSSCR_id
        self.wsscr_users = WSSCR_list



# parsing xml file for lat lng

tree = ET.parse('route.gpx')
root = tree.getroot()

lat_lon_list=[]
for i in root.iter("rtept"):
    lat_lon_list.append(LatLon(i.attrib['lat'],i.attrib['lon']))

#pprint.PrettyPrinter().pprint(lat_lon_list)

    


In [ ]:
def userIdGenerator():
    id=rd.randint(1,1000)
    return "user" + str(id)

def addWSSCR(user,userList):
    radius = WSSCR.cloaked_search_space_radius
    min = float('inf')
    for i in userList:
        diff = round(haversine((user.lat_lon_obj.lat,user.lat_lon_obj.lon),(i.lat_lon_obj.lat,i.lat_lon_obj.lon)),3)
        #print(diff)
        if(diff<radius):
            if(diff<min):
                if(i.wsscr):
                    user.wsscr = i.wsscr
    if(user.wsscr==None):
        user.wsscr = "wsscr:" + str(user.id) +str(rd.randint(1,10))
    
    print(user.wsscr)


userList = []
temp_list = lat_lon_list.copy()
for i in lat_lon_list:
    userLatLon =  rd.choice(temp_list)
    user = UserObject(userIdGenerator(),userLatLon)
    userList.append(user)    
    temp_list.remove(i)

for i in userList:
    addWSSCR(i,userList)
    




In [ ]:
#pprint.PrettyPrinter().pprint(userList)
# for i in userList:
#     print(i.name,i.lat_lon_obj)
    
map = flMap.Map(location=[lat_lon_list[0].lat,lat_lon_list[0].lon], zoom_start=15, tiles="Stamen Terrain")

for i in userList:
    flMap.Marker([i.lat_lon_obj.lat,i.lat_lon_obj.lon],popup=f"{i.lat_lon_obj.lat},{i.lat_lon_obj.lon}",tooltip=f"<i>{i.id}</i>").add_to(map)
    
map



In [77]:
# #pprint.PrettyPrinter().pprint(userList)
# # for i in userList:
# #     print(i.name,i.lat_lon_obj)
    
# map = flMap.Map(location=[lat_lon_list[0].lat,lat_lon_list[0].lon], zoom_start=14, tiles="Stamen Terrain")

# for i in userList:
#     flMap.Marker([i.lat_lon_obj.lat,i.lat_lon_obj.lon],popup=f"{i.wsscr}, {i.lat_lon_obj.lat},{i.lat_lon_obj.lon}",tooltip=f"<i>{i.id}</i>").add_to(map)
    
# map

# wsscr_list=[]

# for i in userList:
#     if(i.wsscr in wsscr_list):
#         continue

#     wsscr_list.append(i.wsscr)
#     flMap.Circle(
#         location=[i.lat_lon_obj.lat,i.lat_lon_obj.lon],
#         radius=1000,
#         popup=i.wsscr,
#         color="#3186cc",
#         fill=True,
#         fill_color="#3186cc",
#     ).add_to(map)

# map


# # Input user  Object
# #lat="30.88321" lon="75.79903"
# UserObjectForTest = UserObject(userIdGenerator(),LatLon(30.88321,75.79903))









In [78]:
#pprint.PrettyPrinter().pprint(userList)
# for i in userList:
#     print(i.name,i.lat_lon_obj)
    
map = flMap.Map(location=[lat_lon_list[0].lat,lat_lon_list[0].lon], zoom_start=14, tiles="Stamen Terrain")

for i in userList:
    flMap.Marker([i.lat_lon_obj.lat,i.lat_lon_obj.lon],popup=f"{i.wsscr}, {i.lat_lon_obj.lat},{i.lat_lon_obj.lon}",tooltip=f"<i>{i.id}</i>").add_to(map)
    
map

wsscr_list=[]

for i in userList:
    if(i.wsscr in wsscr_list):
        continue
    wsscr_list.append(i.wsscr)
    flMap.Circle(
        location=[i.lat_lon_obj.lat,i.lat_lon_obj.lon],
        radius=1000,
        popup=i.wsscr,
        color="#3186cc",
        fill=True,
        fill_color="#3186cc",
    ).add_to(map)

# Input user  Object
#lat="30.87374" lon="75.81251"
#u1 = UserObject(userIdGenerator(),LatLon(30.88321,75.79903))

u1 = UserObject("testing User",LatLon(30.87374,75.81251))

#flMap.Marker([u1.lat_lon_obj.lat,u1.lat_lon_obj.lon],popup=f"{u1.wsscr}, {u1.lat_lon_obj.lat},{u1.lat_lon_obj.lon}",tooltip=f"<i>{u1.id}</i>").add_to(map)

#showing map
map



In [86]:
#implementin knn algorithm
  
# Loading data
temp_list = [[user.id,user.lat_lon_obj.lat,user.lat_lon_obj.lon,user.wsscr] for user in userList]
dataset = pd.DataFrame(temp_list,columns=["User ID","Lat","Lon","WSSCR ID"])

#print(dataset)

# # Create feature and target arrays
x = dataset.iloc[:,1:3].values
y = dataset.iloc[:,-1].values

#print(x)
#print(y)
  
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
  
# # # Predict on dataset which model has not seen before
# print(knn.predict(X_test))



KNeighborsClassifier(n_neighbors=3)

In [87]:

y_pred = knn.predict(X_test)

In [88]:
#print(y_pred)
print(classification_report(y_test,y_pred))
#print(confusion_matrix(y_test,y_pred))

                precision    recall  f1-score   support

wsscr:user3506       1.00      1.00      1.00         1
wsscr:user7738       1.00      1.00      1.00         2
wsscr:user9491       1.00      1.00      1.00         1

      accuracy                           1.00         4
     macro avg       1.00      1.00      1.00         4
  weighted avg       1.00      1.00      1.00         4



In [92]:
data =[[u1.lat_lon_obj.lat,u1.lat_lon_obj.lon]]

predicted_wsscr = knn.predict(data)
u1.wsscr= predicted_wsscr
print(u1.wsscr)


print(wsscr_list)
flMap.Marker([u1.lat_lon_obj.lat,u1.lat_lon_obj.lon],
                icon=flMap.Icon(color='green', icon='info-sign'),
                popup=f"{u1.wsscr}, {u1.lat_lon_obj.lat},{u1.lat_lon_obj.lon}",
                tooltip=f"<i>{u1.id}</i>").add_to(map)
# flMap.Circle(
#         location=[u1.lat_lon_obj.lat,u1.lat_lon_obj.lon],
#         radius=1000,
#         popup=u1.wsscr,
#         color="#3186cc",
#         fill=True,
#         fill_color="#FF0000",
#     ).add_to(map)

map

['wsscr:user3506']
['wsscr:user96', 'wsscr:user3506', 'wsscr:user2981', 'wsscr:user9491', 'wsscr:user7738', 'wsscr:user9308']
